# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

teams_to_metro = {
    'New York Rangers': 'New York City',
    'New York Islanders': 'New York City',
    'New Jersey Devils': 'New York City',
    'Los Angeles Kings': 'Los Angeles',
    'Anaheim Ducks': 'Los Angeles',
    'San Jose Sharks': 'San Francisco Bay Area',
    'Chicago Blackhawks': 'Chicago',
    'Dallas Stars': 'Dallas–Fort Worth',
    'Washington Capitals': 'Washington, D.C.',
    'Philadelphia Flyers': 'Philadelphia',
    'Boston Bruins': 'Boston',
    'Minnesota Wild': 'Minneapolis–Saint Paul',
    'Colorado Avalanche': 'Denver',
    'Florida Panthers': 'Miami–Fort Lauderdale',
    'Arizona Coyotes': 'Phoenix',
    'Detroit Red Wings': 'Detroit',
    'Toronto Maple Leafs': 'Toronto',
    'Tampa Bay Lightning': 'Tampa Bay Area',
    'Pittsburgh Penguins': 'Pittsburgh',
    'Seattle Kraken': 'Seattle',
    'St. Louis Blues': 'St. Louis',
    'Buffalo Sabres': 'Buffalo',
    'Montreal Canadiens': 'Montreal',
    'Vancouver Canucks': 'Vancouver',
    'Columbus Blue Jackets': 'Columbus',
    'Calgary Flames': 'Calgary',
    'Ottawa Senators': 'Ottawa',
    'Edmonton Oilers': 'Edmonton',
    'Winnipeg Jets': 'Winnipeg',
    'Vegas Golden Knights': 'Las Vegas',
    'Carolina Hurricanes': 'Raleigh',
    'Nashville Predators': 'Nashville'
}

def cities():
    cities_var=pd.read_html("assets/wikipedia_data.html")[1]
    cities_var=cities_var.iloc[:-1,[0,3,5,6,7,8]]
    return cities_var

def cities_col(col):
    citiesmlb = cities()[['Metropolitan area','Population (2016 est.)[8]',col]]	
    return citiesmlb

def data_2018(path):
    warnings.filterwarnings("ignore", category=UserWarning)
    df=pd.read_csv(path)
    
    ratio1 = any(df.columns.isin(['W/L%']))
    ratio2 = any(df.columns.isin(['W-L%']))
    
    data_columns = ['year','team', 'W', 'L']
    if ratio1:
        data_columns.extend(['W/L%'])
    elif ratio2:
        data_columns.extend(['W-L%'])
        
    df_2018 = (df.where(df['year'] ==2018).dropna().loc[:, data_columns])
    
    df_2018['year'] = df_2018['year'].astype(int)
    df_2018['team'] = (df_2018['team'].apply(lambda x: re.sub(r'\*', '', x))
                                        .apply(lambda x: re.sub(r"\([0-9]+\)$", '', x))
                                        .apply(lambda x: re.sub(r"\\xa0$", '', x))
                                        .apply(lambda x: re.sub(r'\+$', '', x)))
    
    df_2018= df_2018.where(~df_2018['team'].str.contains(r'[A-Za-z]+\sDivision', regex=True)).dropna()
        
    df_2018['team'] = [name.replace('\xa0', '') for name in df_2018['team'].values]
    
    df_2018= df_2018.reset_index(drop=True)
    warnings.resetwarnings()
    return df_2018


def nhl_2018():
    warnings.filterwarnings("ignore", category=UserWarning)
    
    nhl_2018=data_2018("assets/nhl.csv")
    
    warnings.resetwarnings()
    return nhl_2018


def cities_nhl():
    warnings.filterwarnings("ignore", category=UserWarning)
    
    citiesNHL = cities()[['Metropolitan area','Population (2016 est.)[8]','NHL']]	
    citiesNHL = (citiesNHL.where((citiesNHL['NHL'] != '—') & 
                            (~citiesNHL['NHL'].str.contains(r'(^\[.*\]$)', regex=True)))
                      .dropna()
                      .reset_index(drop=True))
    
    warnings.resetwarnings()
    return citiesNHL

def calc_mean_WL(group):
    group['W'] = group['W'].astype(int)
    group['L'] = group['L'].astype(int)
    w_avg=np.nanmean(group["W"])
    l_avg=np.nanmean(group["L"])
    # now broadcast our formula and create a new column
    group['W']=np.abs(w_avg)
    group['L']=np.abs(l_avg)
    return group

def merged(path, col, dictionary):
    sport = data_2018(path)
    cities_var = cities_col(col)
    
    sport['Metropolitan area'] = sport['team'].map(dictionary)
    
    ratio1 = any(sport.columns.isin(['W/L%']))
    ratio2 = any(sport.columns.isin(['W-L%']))
    
    merge_columns = ['team', 'W', 'L', 'Metropolitan area']
    if ratio1:
        merge_columns.extend(['W/L%'])
    elif ratio2:
        merge_columns.extend(['W-L%'])

    merged_df = pd.merge(cities_var[["Metropolitan area", "Population (2016 est.)[8]"]],
                         sport[merge_columns],
                         on='Metropolitan area', how='left')

    merged_df = merged_df.dropna(subset=['team']).reset_index(drop=True)

    return merged_df

def calc_ratio(group):
    #Win/Loss ratio refers to the number of wins over the number of wins
    #plus the number of losses
    
    group['W'] = group['W'].astype(float)
    group['L'] = group['L'].astype(float)
    ratio=group['W'] / (group['W'] + group['L'])
    
    group['Ratio']=np.abs(ratio)
    return group


def grouped_avg():
    df = merged('assets/nhl.csv','NHL',teams_to_metro)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mean_WL)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_ratio)

    grouped_df = df.groupby('Metropolitan area', as_index=False).agg({
        'Population (2016 est.)[8]': 'first',  # Keep the first value of 'Population'
        'W': 'first',
        'L': 'first',
        'Ratio': 'first',
        'team': ' & '.join  # Concatenate the team names using ' & ' as separator
    })
    return grouped_df



def final_df():
    df = grouped_avg()
    return df

def nhl_correlation(): 
    # YOUR CODE HERE
    df = final_df() 
    
    population_by_region = df['Population (2016 est.)[8]'].values.astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df['Ratio'].values # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    coEff, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return coEff
    raise NotImplementedError()

In [3]:
nhl_correlation()

0.01230899645574425

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

teams_cities_nba = {
    'Toronto Raptors': 'Toronto',
    'Boston Celtics': 'Boston',
    'Philadelphia 76ers': 'Philadelphia',
    'Cleveland Cavaliers': 'Cleveland',
    'Indiana Pacers': 'Indianapolis',
    'Miami Heat': 'Miami–Fort Lauderdale',
    'Milwaukee Bucks': 'Milwaukee',
    'Washington Wizards': 'Washington, D.C.',
    'Detroit Pistons': 'Detroit',
    'Charlotte Hornets': 'Charlotte',
    'New York Knicks': 'New York City',
    'Brooklyn Nets': 'New York City',
    'Chicago Bulls': 'Chicago',
    'Orlando Magic': 'Orlando',
    'Atlanta Hawks': 'Atlanta',
    'Houston Rockets': 'Houston',
    'Golden State Warriors': 'San Francisco Bay Area',
    'Portland Trail Blazers': 'Portland',
    'Oklahoma City Thunder': 'Oklahoma City',
    'Utah Jazz': 'Salt Lake City',
    'New Orleans Pelicans': 'New Orleans',
    'San Antonio Spurs': 'San Antonio',
    'Minnesota Timberwolves': 'Minneapolis–Saint Paul',
    'Denver Nuggets': 'Denver',
    'Los Angeles Clippers': 'Los Angeles',
    'Los Angeles Lakers': 'Los Angeles',
    'Sacramento Kings': 'Sacramento',
    'Dallas Mavericks': 'Dallas–Fort Worth',
    'Memphis Grizzlies': 'Memphis',
    'Phoenix Suns': 'Phoenix'
}


def calc_mean_ratio(group):
    group['W/L%'] = group['W/L%'].astype(float)
    w_avg=np.nanmean(group['W/L%'])
    # now broadcast our formula and create a new column
    group['W/L%']=np.abs(w_avg)
    return group

def grouped_nba():
    df = merged('assets/nba.csv','NBA',teams_cities_nba)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mean_WL)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mean_ratio)
    
    
    grouped_df = df.groupby('Metropolitan area', as_index=False).agg({
        'Population (2016 est.)[8]': 'first',  # Keep the first value of 'Population'
        'W': 'first',
        'L': 'first', 
        'W/L%': 'first',
        'team': ' & '.join  # Concatenate the team names using ' & ' as separator
    })
    return grouped_df


def nba_correlation():
    # YOUR CODE HERE
    df = grouped_nba() 
    
    population_by_region = df['Population (2016 est.)[8]'].values.astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L%'].values # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    coeff, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return coeff
    raise NotImplementedError()

In [5]:
nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

teams_metro_mlb = {
    'Boston Red Sox': 'Boston',
    'New York Yankees': 'New York City',
    'Tampa Bay Rays': 'Tampa Bay Area',
    'Toronto Blue Jays': 'Toronto',
    'Baltimore Orioles': 'Baltimore',
    'Cleveland Indians': 'Cleveland',
    'Minnesota Twins': 'Minneapolis–Saint Paul',
    'Detroit Tigers': 'Detroit',
    'Chicago White Sox': 'Chicago',
    'Kansas City Royals': 'Kansas City',
    'Houston Astros': 'Houston',
    'Oakland Athletics': 'San Francisco Bay Area',
    'Seattle Mariners': 'Seattle',
    'Los Angeles Angels': 'Los Angeles',
    'Texas Rangers': 'Dallas–Fort Worth',
    'Atlanta Braves': 'Atlanta',
    'Washington Nationals': 'Washington, D.C.',
    'Philadelphia Phillies': 'Philadelphia',
    'New York Mets': 'New York City',
    'Miami Marlins': 'Miami–Fort Lauderdale',
    'Milwaukee Brewers': 'Milwaukee',
    'Chicago Cubs': 'Chicago',
    'St. Louis Cardinals': 'St. Louis',
    'Pittsburgh Pirates': 'Pittsburgh',
    'Cincinnati Reds': 'Cincinnati',
    'Los Angeles Dodgers': 'Los Angeles',
    'Colorado Rockies': 'Denver',
    'Arizona Diamondbacks': 'Phoenix',
    'San Francisco Giants': 'San Francisco Bay Area',
    'San Diego Padres': 'San Diego'
}

def calc_mlb_ratio(group):
    group['W-L%'] = group['W-L%'].astype(float)
    w_avg=np.nanmean(group['W-L%'])
    # now broadcast our formula and create a new column
    group['W-L%']=np.abs(w_avg)
    return group


def grouped_mlb():
    df = merged('assets/mlb.csv','MLB',teams_metro_mlb)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mlb_ratio)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mean_WL)
    
    grouped_df = df.groupby('Metropolitan area', as_index=False).agg({
        'Population (2016 est.)[8]': 'first',  # Keep the first value of 'Population'
        'W': 'first',
        'L': 'first', 
        'W-L%': 'first',
        'team': ' & '.join  # Concatenate the team names using ' & ' as separator
    })
    return grouped_df

def mlb_correlation(): 
    # YOUR CODE HERE
    df = grouped_mlb() 
    
    population_by_region = df['Population (2016 est.)[8]'].values.astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'].values # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    coeff, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return coeff
    raise NotImplementedError()

In [7]:
mlb_correlation()

0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

teams_metro_nfl = {
    'New England Patriots': 'Boston',
    'Miami Dolphins': 'Miami–Fort Lauderdale',
    'Buffalo Bills': 'Buffalo',
    'New York Jets': 'New York City',
    'Baltimore Ravens': 'Baltimore',
    'Pittsburgh Steelers': 'Pittsburgh',
    'Cleveland Browns': 'Cleveland',
    'Cincinnati Bengals': 'Cincinnati',
    'Houston Texans': 'Houston',
    'Indianapolis Colts': 'Indianapolis',
    'Tennessee Titans': 'Nashville',
    'Jacksonville Jaguars': 'Jacksonville',
    'Kansas City Chiefs': 'Kansas City',
    'Los Angeles Chargers': 'Los Angeles',
    'Denver Broncos': 'Denver',
    'Oakland Raiders': 'San Francisco Bay Area',
    'Dallas Cowboys': 'Dallas–Fort Worth',
    'Philadelphia Eagles': 'Philadelphia',
    'Washington Redskins': 'Washington, D.C.',
    'New York Giants': 'New York City',
    'Chicago Bears': 'Chicago',
    'Minnesota Vikings': 'Minneapolis–Saint Paul',
    'Green Bay Packers': 'Green Bay',
    'Detroit Lions': 'Detroit',
    'New Orleans Saints': 'New Orleans',
    'Carolina Panthers': 'Charlotte',
    'Atlanta Falcons': 'Atlanta',
    'Tampa Bay Buccaneers': 'Tampa Bay Area',
    'Los Angeles Rams': 'Los Angeles',
    'Seattle Seahawks': 'Seattle',
    'San Francisco 49ers': 'San Francisco Bay Area',
    'Arizona Cardinals': 'Phoenix'}


def nfl_grouped():
    df = merged('assets/nfl.csv','NFL',teams_metro_nfl)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mlb_ratio)
    df = df.groupby('Metropolitan area',group_keys=False).apply(calc_mean_WL)
    
    grouped_df = df.groupby('Metropolitan area', as_index=False).agg({
        'Population (2016 est.)[8]': 'first',  # Keep the first value of 'Population'
        'W': 'first',
        'L': 'first', 
        'W-L%': 'first',
        'team': ' & '.join  # Concatenate the team names using ' & ' as separator
    })
    return grouped_df
    

def nfl_correlation(): 
    # YOUR CODE HERE
    df = nfl_grouped()
    
    population_by_region = df['Population (2016 est.)[8]'].values.astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'].values # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    coeff, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return coeff
    raise NotImplementedError()


In [9]:
#Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses.

nfl_grouped()

,Metropolitan area,Population (2016 est.)[8],W,L,W-L%,team
0,Atlanta,5789700,7.0,9.0,0.4380,Atlanta Falcons
1,Baltimore,2798886,10.0,6.0,0.6250,Baltimore Ravens
2,Boston,4794447,11.0,5.0,0.6880,New England Patriots
3,Buffalo,1132804,6.0,10.0,0.3750,Buffalo Bills
4,Charlotte,2474314,7.0,9.0,0.4380,Carolina Panthers
5,Chicago,9512999,12.0,4.0,0.7500,Chicago Bears
6,Cincinnati,2165139,6.0,10.0,0.3750,Cincinnati Bengals
7,Cleveland,2055612,7.0,8.0,0.4690,Cleveland Browns
8,Dallas–Fort Worth,7233323,10.0,6.0,0.6250,Dallas Cowboys
9,Denver,2853077,6.0,10.0,0.3750,Denver Broncos


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [47]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
from itertools import combinations
import re

paired_t_test_results = {}
nhl = final_df()[['Metropolitan area','team','Ratio']]
nhl.rename(columns={'Ratio':'W-L%'},inplace=True)

nba = grouped_nba()[['Metropolitan area','team','W/L%']]
nba.rename(columns={'W/L%':'W-L%'},inplace=True)

mlb = grouped_mlb()[['Metropolitan area','team','W-L%']]
nfl = nfl_grouped()[['Metropolitan area','team','W-L%']]

sports_dfs = [nhl, nba, mlb, nfl]

def paired_t_test(df1, df2):
    t_statistic, p_value = ttest_rel(df1, df2)
    return p_value

def get_df_name(dataframe):
    return [name for name, obj in locals().items() if obj is dataframe][0]


def sports_team_performance():
    # YOUR CODE HERE
    sports_dfs = [nfl, nba, nhl, mlb]
    sports = ['NFL','NBA','NHL','MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in range(len(sports_dfs)):
        for j in range(i+1, len(sports_dfs)):
            sport1_name = sports[i]
            sport2_name = sports[j]
            sport1_df = sports_dfs[i]
            sport2_df = sports_dfs[j]
            mtype = 'inner'
            
            merged_df = pd.merge(sport1_df[['Metropolitan area', 'team', 'W-L%']],
                             sport2_df[['Metropolitan area', 'team', 'W-L%']],
                             on=['Metropolitan area'],
                             how=mtype)
            
            merged_df.rename(columns = {'W-L%_x':sport1_name,'W-L%_y':sport2_name},inplace=True)
            print(sport1_name, sport2_name, len(merged_df))
            
            
            # Conduct paired t-test
            p_value = paired_t_test(merged_df[sport1_name], merged_df[sport2_name])
            #print (sport1_name, sport2_name, p_value)
            p_values.loc[sport1_name, sport2_name] = p_value
            p_values.loc[sport2_name, sport1_name] = p_value

    # Convert the dictionary of results to a pandas Series
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    raise NotImplementedError()

    


In [48]:
sports_team_performance()
#sports_team_performance2(nhl, nba, mlb, nfl)

NFL NBA 19
NFL NHL 17
NFL MLB 22
NBA NHL 14
NBA MLB 18
NHL MLB 17


,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030392,0.803459
NBA,0.937509,NaN,0.022405,0.949566
NHL,0.030392,0.022405,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
